In [1]:
# Dependencies
import json
import time
import os
import requests
import twitter
import numpy as np

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
# Loading Keys
api_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath('__file__'))))))
data = json.load(open(os.path.join(api_dir, "api_keys.json")))

# Setting Keys
consumer_key = data['twitter_consumer_key']
consumer_secret = data['twitter_consumer_secret']
access_token = data['twitter_access_token']
access_token_secret = data['twitter_access_token_secret']

In [3]:
# Setting up Auth
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token,
                  access_token_secret=access_token_secret)

In [4]:
# Tweet Gather function
def tweetGrab(city, city_dict, start_time):

    # Call to API - each new call should return new tweets and count against rate-limit
    tweets = api.GetSearch(geocode=[city_dict[f'{city}']['gps'][1],city_dict[f'{city}']['gps'][0], '20mi'],return_json=True)

    # Loop to parse tweet and append data needed
    for tweet in tweets['statuses']:

        # If loop to ensure unique tweets
        if tweet['text'] in city_dict[f'{city}']['data']['text']:
            continue
        else:        
            # Appending important data
            city_dict[f'{city}']['data']['text'].append(tweet['text'])
            city_dict[f'{city}']['data']['bounding_box'].append(solveBox(tweet['place']['bounding_box']['coordinates']))
            city_dict[f'{city}']['data']['user'].append(tweet['user']['screen_name'])
            city_dict[f'{city}']['data']['created_at'].append(tweet['created_at'])
            city_dict[f'{city}']['data']['followers_count'].append(tweet['user']['followers_count'])
            city_dict[f'{city}']['data']['comp_sent'].append(analyzer.polarity_scores(tweet['text'])['compound'])

            # Creating try loops for variable data that may or may not show up
            try:
                city_dict[f'{city}']['data']['coords'].append(tweet['coordinates']['coordinates'])
            except:
                city_dict[f'{city}']['data']['coords'].append(tweet['coordinates'])

            try:
                city_dict[f'{city}']['data']['profile_image_url'].append(tweet['user']['profile_image_url'])
            except:
                city_dict[f'{city}']['data']['profile_image_url'].append('None')

            # Printing status of calls and current time
            if (len(city_dict[f'{city}']['data']['text'])%5 == 0):
                print(f'{city}: ' + str(len(city_dict[f'{city}']['data']['text'])) + ' --- %s seconds' % round(time.clock() - start_time,2))    

    # Ending function
    return()

In [5]:
# Calculating a rough coord based on bounding box
def solveBox(bounding_box):
    
    # Creating temp dict and variable to return
    coord_dict = {'lat': [], 'lon': []}
    coord_return = []

    # Small loop to append each coord to it's own list to sum
    for coord in bounding_box[0]:
        coord_dict['lat'].append(coord[1] + (np.random.rand() - np.random.rand()))
        coord_dict['lon'].append(coord[0] + (np.random.rand() - np.random.rand()))

    # Appending the sums to a return list
    coord_return.append(round(np.mean(coord_dict['lon']),6))
    coord_return.append(round(np.mean(coord_dict['lat']),6))

    # Return list of coords
    return(coord_return)

In [6]:
# Fills out the dictionary
def fillDict():
    
    # City dictionary with static coords
    city_dict = {
        'Dallas': {'gps':[-96.796988, 32.776664], 'data':{}},
        'St. Louis': {'gps':[-90.199404, 38.627003], 'data':{}},
        'Los Angeles': {'gps':[-118.243685, 34.052234], 'data':{}},
        'Atlanta': {'gps':[-84.387982, 33.748995], 'data':{}},
        'Chicago': {'gps':[-87.629798, 41.878114], 'data':{}},
        'Miami': {'gps':[-80.191790, 25.761680], 'data':{}},
        'New York': {'gps':[-74.005973, 40.712775], 'data':{}},
        'Kansas City': {'gps':[-94.578567, 39.099727], 'data':{}},
        'Seattle': {'gps':[-122.332071, 47.606210], 'data':{}},
        'Las Vegas': {'gps':[-115.139830, 36.169941], 'data':{}}
        }

    # Adding template to dictionary
    for city in city_dict:

        # Making lists for each entry
        city_dict[f'{city}']['data']['text'] = [] 
        city_dict[f'{city}']['data']['coords'] = []
        city_dict[f'{city}']['data']['bounding_box'] = []
        city_dict[f'{city}']['data']['user'] = []
        city_dict[f'{city}']['data']['profile_image_url'] = []
        city_dict[f'{city}']['data']['created_at'] = []
        city_dict[f'{city}']['data']['comp_sent'] = []
        city_dict[f'{city}']['data']['followers_count'] = []

    # Returning the dictionary
    return(city_dict)

In [7]:
# Function to compress limit check
def limit_check(check, city_dict, tweet_goal):
    
    # Creating empty check list
    check_arr = []
    
    # Checking if any of the lists have reached the limit
    if (check == 'first_to_goal'):
        for city in city_dict:
            if (len(city_dict[f'{city}']['data']['text'])>=tweet_goal):
                check_arr.append(True)
            else:
                check_arr.append(False)
                
        # Setting limit based on 'not any' logic
        limit = not any(check_arr)
    
    # Checking if all the lists have reached the limit
    if (check == 'all_to_goal'):
        for city in city_dict:
            if (len(city_dict[f'{city}']['data']['text'])<tweet_goal):
                check_arr.append(True)
            else:
                check_arr.append(False)   
        
        # Setting limit based on 'any' logic
        limit = any(check_arr)
    
    # Returning a bool to check against for loop
    return(limit)

In [8]:
# Save output to a txt/json doc for easy read later
def saveOutput(data, tweet_goal, start_time, limit_type):
    with open('data.txt', 'w') as outfile:
        json.dump(data, outfile, sort_keys = True, indent = 2)
    with open(f'{limit_type} - {tweet_goal} Tweets - ' + str(round(time.clock()-start_time,2)) + f' Runtime - {start_time}.txt', 'w') as outfile:
        json.dump(data, outfile, sort_keys = True, indent = 2)

In [9]:
# Print Final Output
def printOutput(city_dict, start_time):
    
    # Starting printing format
    print('-------------------------------------------')
    
    # Loop to print each list length
    for city in city_dict:
        print(f'{city}:' + str(len(city_dict[f'{city}']['data']['text'])))
    
    # Final Format Print with runtime
    print('-------------------------------------------')
    print(' --- %s Runtime' % round(time.clock() - start_time,2))

In [10]:
# Main Execution
def twitterize():
    
    # Setting up statics
    tweet_goal = 20
    limit_type = 'first_to_goal' # 'first_to_goal' or 'all_to_goal'
    start_time = time.clock()
    
    # Finding the dict
    city_dict = fillDict()

    # Main loop - Checks what type of limit is set and runs until False is returned
    while (limit_check(limit_type, city_dict, tweet_goal) == True):
        
        # Rotates cities - Causes a delay so new tweets can be fed to the API
        for city in city_dict:
            tweetGrab(city, city_dict, start_time)
            time.sleep(6)

    # Saving output to txt file
    saveOutput(city_dict, tweet_goal, start_time, limit_type)
    
    # Printing runtime
    printOutput(city_dict, start_time)

In [11]:
twitterize()

Kansas City: 5 --- 170.44 seconds
Dallas: 5 --- 189.71 seconds
Chicago: 5 --- 277.51 seconds
Kansas City: 10 --- 296.29 seconds
Las Vegas: 5 --- 371.76 seconds
Los Angeles: 5 --- 390.64 seconds
Miami: 5 --- 409.46 seconds
New York: 5 --- 415.78 seconds
Seattle: 5 --- 428.29 seconds
Dallas: 10 --- 440.77 seconds
St. Louis: 5 --- 509.99 seconds
Chicago: 10 --- 591.57 seconds
Atlanta: 5 --- 711.05 seconds
Dallas: 15 --- 755.28 seconds
Los Angeles: 10 --- 767.91 seconds
Miami: 10 --- 786.67 seconds
New York: 10 --- 792.93 seconds
Seattle: 10 --- 805.47 seconds
Las Vegas: 10 --- 811.72 seconds
Chicago: 15 --- 843.06 seconds
Kansas City: 15 --- 861.89 seconds
St. Louis: 10 --- 949.79 seconds
Dallas: 20 --- 1006.12 seconds
Miami: 15 --- 1037.72 seconds
New York: 15 --- 1044.0 seconds
Kansas City: 20 --- 1050.24 seconds
-------------------------------------------
Dallas:21
St. Louis:10
Los Angeles:14
Atlanta:9
Chicago:18
Miami:15
New York:15
Kansas City:20
Seattle:13
Las Vegas:14
-------------